In [ ]:
!pip install gensim
!pip install compress-fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 16.0 MB/s eta 0:00:00
  Created wheel for compress-fasttext: filename=compress_fasttext-0.1.3-py3-none-any.whl size=14601 sha256=5cefa17b23bc5586b66adee1e261bbc030e00ef8a6e5171306a22583c815bb91
  Stored in directory: /root/.cache/pip/wheels/4d/c6/5a/47903dfee0ac92e78775d4364427259b483ba860be86445b05
Successfully built compress-fasttext
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import time
import torch
import re
import random as r

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os

In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/DOM')

## Загрузка данных

In [ ]:
df_e = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/DOM/NETOLOGIA/simpsons_script_lines.csv').dropna()
print(df_e.shape)

(131708, 13)


/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_e.head()

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40.0,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33


In [ ]:
df = df_e.drop(["spoken_words", "id", "episode_id", "number", "timestamp_in_ms" ,"speaking_line", "character_id", "location_id", "raw_text", "raw_location_text"], axis=1)
df.head()

,raw_character_text,normalized_text,word_count
0,Miss Hoover,no actually it was a little of both sometimes ...,31
1,Lisa Simpson,wheres mr bergstrom,3
2,Miss Hoover,i dont know although id sure like to talk to h...,22
3,Lisa Simpson,that life is worth living,5
4,Edna Krabappel-Flanders,the polls will be open from now until the end ...,33


In [ ]:
df.shape

(131708, 3)

In [ ]:
df.iloc[4,1]

'the polls will be open from now until the end of recess now just in case any of you have decided to put any thought into this well have our final statements martin'

In [ ]:
df[df['raw_character_text'] == 'Bart Simpson'].shape, df[df['raw_character_text'] == 'Lisa Simpson'].shape

((12997, 3), (10751, 3))

In [ ]:
from gensim import models

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(x, y):
    return dot(x, y) / (norm(x) * norm(y))

print(cos_sim([1, 0, 0], (1, 0, 0)))
print(cos_sim([1, 0, 0], (0, 1, 0)))
print(cos_sim([1, 0, 0], (-1, 0, 0)))

1.0
0.0
-1.0


In [ ]:
sentences = []
for i in df['normalized_text']:
  sentences.append(i.split(' '))
#for i in lst:
#  for j in i:
#    sentences.append(j)

In [ ]:
sentences[:2]

[['no',
  'actually',
  'it',
  'was',
  'a',
  'little',
  'of',
  'both',
  'sometimes',
  'when',
  'a',
  'disease',
  'is',
  'in',
  'all',
  'the',
  'magazines',
  'and',
  'all',
  'the',
  'news',
  'shows',
  'its',
  'only',
  'natural',
  'that',
  'you',
  'think',
  'you',
  'have',
  'it'],
 ['wheres', 'mr', 'bergstrom']]

In [ ]:
len(sentences)

131708

#### Обучим модель word2vec

In [ ]:
model = models.Word2Vec()
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(4760691, 6510365)

In [ ]:
try:
    print(cos_sim(model.wv['actually'], model.wv['little']))
    print(cos_sim(model.wv['stay'], model.wv['stand']))
    print(cos_sim(model.wv['natural'], model.wv['naturval']))
except Exception as e:
    print(e)

0.22056991
0.78270984
"Key 'naturval' not present"




---



 homer - marge + bart

 bart - lisa + school

 marge - homer + home

In [ ]:
model.wv.most_similar(positive=["homer", "bart"], negative=["marge"], topn=5)

[('lisa', 0.737775981426239),
 ('grampa', 0.6546167135238647),
 ('abe', 0.6112764477729797),
 ('mrs', 0.6064900755882263),
 ('milhouse', 0.6044096350669861)]

In [ ]:
model.wv.most_similar(positive=["bart", "school"], negative=["lisa"], topn=5)

[('church', 0.7087029218673706),
 ('store', 0.6599953174591064),
 ('game', 0.643497109413147),
 ('christmas', 0.6415625810623169),
 ('house', 0.6401897072792053)]

In [ ]:
model.wv.most_similar(positive=["marge", "home"], negative=["homer"], topn=5)

[('back', 0.7288848757743835),
 ('sleep', 0.6630720496177673),
 ('bed', 0.6360216736793518),
 ('together', 0.5854783654212952),
 ('away', 0.5525188446044922)]

In [ ]:
model.wv.similarity("lisa", 'bart')

0.89721406



---



#### Обучим модель FastText

In [ ]:
model = models.FastText(sentences, sg=1, vector_size=50)

In [ ]:
try:
    print(cos_sim(model.wv['actually'], model.wv['little']))
    print(cos_sim(model.wv['stay'], model.wv['stand']))
    print(cos_sim(model.wv['natural'], model.wv['naturval']))
except Exception as e:
    print(e)

0.4349786
0.79278976
0.97424173


#### Загрузим оптимизированную модель FastText

In [ ]:
import compress_fasttext
model = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    'https://github.com/avidale/compress-fasttext/releases/download/gensim-4-draft/geowac_tokens_sg_300_5_2020-100K-20K-100.bin'
)

In [ ]:
try:
    print(cos_sim(model['actually'], model['little']))
    print(cos_sim(model['stay'], model['stand']))
    print(cos_sim(model['natural'], model['naturval']))
except Exception as e:
    print(e)

0.63837904
0.68211174
0.840674


In [ ]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []


fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)

for w in fd.most_common(1000):
    top_words.append(w[0])

print(top_words[:50:])
top_words_vec = model[top_words]

<ipython-input-28-c8c3c09dbfcf>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm(sentences):


  0%|          | 0/131708 [00:00<?, ?it/s]

['the', 'you', 'i', 'a', 'to', 'and', 'of', 'it', 'that', 'my', 'in', 'is', 'this', 'me', 'your', 'for', 'im', 'we', 'on', 'oh', 'what', 'have', 'no', 'but', 'be', 'well', 'its', 'dont', 'all', 'just', 'with', 'do', 'are', 'not', 'now', 'like', 'so', 'was', 'get', 'youre', 'can', 'know', 'one', 'at', 'thats', 'up', 'out', 'here', 'hey', 'if']


In [ ]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 13.2 s, sys: 286 ms, total: 13.5 s
Wall time: 14.9 s


In [ ]:
!pip install bokeh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)